In [1]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader
from surprise.model_selection import KFold, cross_validate, train_test_split
from tabulate import tabulate

# Create a KNN Ranking algorithm version

In [2]:
from surprise import AlgoBase
from surprise import Dataset
from six import iteritems
import heapq
from surprise.prediction_algorithms.predictions import PredictionImpossible
from sklearn.preprocessing import MinMaxScaler

class SymmetricAlgo(AlgoBase):
    """
    This is an abstract class aimed to ease the use of symmetric algorithms.
    A symmetric algorithm is an algorithm that can can be based on users or on
    items indifferently. When the algo is user-based x denotes a user and y an item. 
    Else, it's reversed.
    """
    def __init__(self, sim_options={}, verbose=True, **kwargs):

        AlgoBase.__init__(self, sim_options=sim_options, **kwargs)
        self.verbose = verbose

    def fit(self, trainset):

        AlgoBase.fit(self, trainset)

        ub = self.sim_options['user_based']
        self.n_x = self.trainset.n_users if ub else self.trainset.n_items
        self.n_y = self.trainset.n_items if ub else self.trainset.n_users
        self.xr = self.trainset.ur if ub else self.trainset.ir
        self.yr = self.trainset.ir if ub else self.trainset.ur

        return self

    def switch(self, u_stuff, i_stuff):
        """
        Return x_stuff and y_stuff depending on the user_based field.
        """

        if self.sim_options['user_based']:
            return u_stuff, i_stuff
        else:
            return i_stuff, u_stuff


class RankingKNNBasic(SymmetricAlgo):
    def __init__(self, k=40, min_k=1, sim_options={}, verbose=True, **kwargs):

        SymmetricAlgo.__init__(self, sim_options=sim_options, verbose=verbose,
                               **kwargs)
        self.k = k
        self.min_k = min_k

    def fit(self, trainset):

        SymmetricAlgo.fit(self, trainset)
        self.sim = self.compute_similarities()

        return self

    def estimate(self, u, i):
        # Penalize items which are unknown, don't just skip them fromt the computation
        self.unknown_considerations = 0

        if not (self.trainset.knows_user(u) or self.trainset.knows_item(i)):
            self.unknown_considerations += 1
            
        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible('User and/or item is unknown.')

        x, y = self.switch(u, i)
        
        neighbors = [(self.sim[x, x2], r) for (x2, r) in self.yr[y]]
        k_neighbors = heapq.nlargest(self.k, neighbors, key=lambda t: t[0])
        
        # compute only the sum of similarities (ranking)
        sum_sim = actual_k = 0
        sim_sums = []
        for (sim, r) in k_neighbors:
            sum_sim += sim
            actual_k += 1
            sim_sums.append(sum_sim)

        if actual_k < self.min_k:
            raise PredictionImpossible('Not enough neighbors.')
        
        est = sum_sim 
        
        details = {'actual_k': actual_k, 'unknown_items': self.unknown_considerations}
        return est, details

# Define a way to get precision and recall

In [3]:
from collections import defaultdict

def precision_recall_at_k(predictions, trainset_user_ratings, k=5, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    
    for uid, iid, true_r, est, details in predictions:
        user_est_true[uid].append((est, true_r, details, iid))
    
    precisions = dict()
    recalls = dict()    
    
    for uid, user_ratings in user_est_true.items():
        
        # nr_uknowns 
        nr_unknowns = user_ratings[0][2]
        nr_unknowns = nr_unknowns.get('unknown_items')
        if nr_unknowns == None:
            nr_unknowns = 0
        
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r, _ , _) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _ , _ , _) in user_ratings[:k])

        # relevant and recommended items in top k
        rel_and_rec_k = [(est, true_r, iid) for (est, true_r, _, iid) in user_ratings[:k] if ((true_r >= threshold) and (est >= threshold))]
        
        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = len(rel_and_rec_k)
        
        # Number of recommended items which the user has already rated
        # Penalize recommendations for items that are already in the train set for said user
        for recommended in rel_and_rec_k:
            for already_seen in trainset.ur[uid]:
                if recommended[2] == already_seen[0]:
                    n_rel_and_rec_k -= 1
                    nr_unknowns += 1

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / (k + nr_unknowns)

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / (n_rel + nr_unknowns) if n_rel != 0 else 0


    return precisions, recalls

# Read and load data

In [4]:
# Read both the datasets from the files using pandas
movielens_df = pd.read_csv("../data/u.data", sep="\t", header=None)
movielens_df.columns = ["userID", "itemID", "rating", "timestamp"]
pda_df = pd.read_csv("../data/train-PDA2018.csv", sep=",")
print(movielens_df.head())
print("\n\n")
print(pda_df.head())

   userID  itemID  rating  timestamp
0     196     242       3  881250949
1     186     302       3  891717742
2      22     377       1  878887116
3     244      51       2  880606923
4     166     346       1  886397596



   userID  itemID  rating  timeStamp
0       5     648       5  978297876
1       5    1394       5  978298237
2       5    3534       5  978297149
3       5     104       4  978298558
4       5    2735       5  978297919


In [5]:
# Create the training datasets using Surprise's reader class
reader = Reader(rating_scale=(1,5)) # We have ratings from 1 to 5 so we create the rating scale

# Load the data from the dataframes
movielens_dataset = Dataset.load_from_df(movielens_df.iloc[:,0:3], reader)
pda_dataset = Dataset.load_from_df(pda_df.iloc[:,0:3], reader)

# Build full trainsets to print out the data loaded above
# mls_train = movielens_dataset.build_full_trainset()
# pda_train = pda_dataset.build_full_trainset()
mls_train, mls_test = train_test_split(data=movielens_dataset, test_size=0.2)
pda_train, pda_test = train_test_split(data=pda_dataset, test_size=0.2)

# Print out some basic information about the datasets
print("General information on the training sets we will be using \n")
print("1) Number of items in each dataset", " ML100k:", mls_train.n_items, "PDA:", pda_train.n_items)
print("2) Number of users in each dataset", " ML100k:", mls_train.n_users, "PDA:", pda_train.n_users)
print("3) Number of ratings in each dataset", " ML100k:", mls_train.n_ratings, "PDA:", pda_train.n_ratings)
print("4) Mean rating", " ML100k:", mls_train.global_mean, "PDA:", pda_train.global_mean)

General information on the training sets we will be using 

1) Number of items in each dataset  ML100k: 1646 PDA: 1822
2) Number of users in each dataset  ML100k: 943 PDA: 5680
3) Number of ratings in each dataset  ML100k: 80000 PDA: 376568
4) Mean rating  ML100k: 3.5279875 PDA: 3.637459901000616


In [6]:
kf = KFold(2)
fold_nr = 1
user_algo = RankingKNNBasic(k=80, sim_options={'name': 'cosine', 'user_based': True})

for trainset, testset in kf.split(movielens_dataset):
    print("Fold number", fold_nr)
    user_algo.fit(trainset)
    predictions = user_algo.test(testset)
    print(predictions[:2])
    pres_5, recs_5 = precision_recall_at_k(predictions, trainset.ur, k=5)
    pres_10, recs_10 = precision_recall_at_k(predictions, trainset.ur, k=10)
    
    # Precision and recall averaged over all users for this fold
    pre_5 = np.array([pres_5[k] for k in pres_5.keys()]).mean()
    rec_5 = np.array([recs_5[k] for k in recs_5.keys()]).mean()
    pre_10 = np.array([pres_10[k] for k in pres_10.keys()]).mean()
    rec_10 = np.array([recs_10[k] for k in recs_10.keys()]).mean()

    print("Precision and Recall @5:", pre_5, rec_5)
    print("Precision and Recall @10:", pre_10, rec_10)
    fold_nr += 1
    print()

Fold number 1
Computing the cosine similarity matrix...
Done computing similarity matrix.
[Prediction(uid=200, iid=515, r_ui=5.0, est=5, details={'actual_k': 80, 'unknown_items': 0, 'was_impossible': False}), Prediction(uid=42, iid=627, r_ui=2.0, est=5, details={'actual_k': 40, 'unknown_items': 0, 'was_impossible': False})]
Precision and Recall @5: 0.5357407968489623 0.18917065162084593
Precision and Recall @10: 0.5222174079013951 0.35708878701143293

Fold number 2
Computing the cosine similarity matrix...
Done computing similarity matrix.
[Prediction(uid=276, iid=411, r_ui=4.0, est=5, details={'actual_k': 80, 'unknown_items': 0, 'was_impossible': False}), Prediction(uid=328, iid=127, r_ui=5.0, est=5, details={'actual_k': 80, 'unknown_items': 0, 'was_impossible': False})]
Precision and Recall @5: 0.5416325809220824 0.18261461747527719
Precision and Recall @10: 0.5328828470928153 0.35480548513953514



In [7]:
fold_nr = 1
item_algo = RankingKNNBasic(k=80, sim_options={'name': 'cosine', 'user_based': False})

for trainset, testset in kf.split(movielens_dataset):
    print("Fold number", fold_nr)
    item_algo.fit(trainset)
    predictions = item_algo.test(testset)
    pres_5, recs_5 = precision_recall_at_k(predictions, trainset.ur, k=5)
    pres_10, recs_10 = precision_recall_at_k(predictions, trainset.ur, k=10)
    
    # Precision and recall averaged over all users for this fold
    pre_5 = np.array([pres_5[k] for k in pres_5.keys()]).mean()
    rec_5 = np.array([recs_5[k] for k in recs_5.keys()]).mean()
    pre_10 = np.array([pres_10[k] for k in pres_10.keys()]).mean()
    rec_10 = np.array([recs_10[k] for k in recs_10.keys()]).mean()

    print("Precision and Recall @5:", pre_5, rec_5)
    print("Precision and Recall @10:", pre_10, rec_10)
    fold_nr += 1
    print()

Fold number 1
Computing the cosine similarity matrix...
Done computing similarity matrix.
Precision and Recall @5: 0.5228298742614755 0.18602214898830088
Precision and Recall @10: 0.5197578314227307 0.3568674805289799

Fold number 2
Computing the cosine similarity matrix...
Done computing similarity matrix.
Precision and Recall @5: 0.5293457220286488 0.18673974609498895
Precision and Recall @10: 0.5255943596876789 0.3574375039241169

